In [ ]:
#!python run.py

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import qpfas

In [ ]:
exp_df = pd.read_json("results_under30Q.json")
exp_df

In [ ]:
def nice_name(x):
    num_idx = [i for i in range(len(x)) if x[i].isnumeric()]
    for c, i in enumerate(num_idx):
        x = x[:i+c] + "_" + x[i+c:]
    x = "$" + x + "$"
    return x

In [ ]:
full_active_name = {"None": "Full", 
                    "frozen_core": "Frozen Core", 
                    "noons_0.01":"NOONs (0.01)", 
                    "noons_0.005":"NOONs (0.005)", 
                    "noons_0.002":"NOONs (0.002)"}

In [ ]:
benchmark_method = "ccsd"


mols = [i["molecule"] for _, i in exp_df[exp_df["active_space_method"] == "None"].iterrows()]
print(mols)

width = 0.2
rs = np.arange(len(mols))

fig, axs = plt.subplots(nrows=3, sharex="col", figsize=(8.4, 5))


# ax 1: error
exact_energy = exp_df[exp_df["active_space_method"]=="None"][benchmark_method + "_energy"].values
print(exact_energy)

for c, i in enumerate(["None", "frozen_core", "noons_0.002"]):
    """
    1) Error in Energy
    """
    # hf error
    if i == "None":
        hf_err = exp_df[exp_df["active_space_method"]==i]["hf_energy"].values - exact_energy
        axs[0].bar(rs-width, hf_err, width=width, label='HF', color='C3', edgecolor='k')

    else:
        err = exp_df[exp_df["active_space_method"]==i][benchmark_method + "_energy"].values - exact_energy
        axs[0].bar(rs+width*(c-1), err, width=width, label=full_active_name[i], color='C%i' % c, edgecolor='k')
        print(err)

    """
    2) Number of Qubits    
    """
    n_qubits = [i["gate_stats_he"]["n_qubits"] for _, i in exp_df[exp_df["active_space_method"] == i].iterrows()]
    n_qubits2 = [i["gate_stats_uccsd"]["n_qubits"] for _, i in exp_df[exp_df["active_space_method"] == i].iterrows()]

    print(n_qubits, n_qubits2)

    if i == "None":
        axs[1].bar(rs+width*(c-1), n_qubits, width=width, color='C%i' % c, edgecolor='k', label=full_active_name[i])
    else:
        axs[1].bar(rs+width*(c-1), n_qubits, width=width, color='C%i' % c, edgecolor='k')

    n_qubits_taper = [i["taper_output"]["n_qubits_taper"] for _, i in exp_df[exp_df["active_space_method"] == i].iterrows()]
    axs[1].bar(rs+width*(c-1), n_qubits_taper, width=width, color='C%i' % c, hatch="////", edgecolor='k')
    
    """
    3) Number of Ham. Terms    
    """
    n_terms = exp_df[exp_df["active_space_method"]==i]["n_hamiltonian_terms"].values
    axs[2].bar(rs+width*(c-1), n_terms, width=width, color='C%i' % c, edgecolor='k')
    n_terms_taper = [i["taper_output"]["n_hamiltonian_terms_taper"] for _, i in exp_df[exp_df["active_space_method"] == i].iterrows()]
    axs[2].bar(rs+width*(c-1), n_terms_taper, width=width, color='C%i' % c, hatch="////", edgecolor='k')

axs[0].set_ylabel("Error (H)")
axs[1].set_ylabel("Num. of Qubits")
axs[2].set_ylabel("Num. of Terms")

axs[0].legend()
axs[1].legend()

axs[0].set_yscale("log")
axs[2].set_yscale("log")

names = [nice_name(a) for a in mols]
plt.xticks(rs, names)
plt.savefig("cr_qubit_reduction.pdf")
plt.show()



In [ ]:
ansatz_name_sl = {"he":"Hardware Eff.", "uccsd":"UCCSD"}

In [ ]:
mol = "CH3F"
df_sub = exp_df[exp_df["molecule"] == mol]

gates = ["gate_total", "gate_cx", "gate_rx", "gate_ry", "gate_rz"]

width = 0.1

fig, axs = plt.subplots(nrows=2, sharex='col')

as_method = df_sub["active_space_method"]
print(as_method)

xs = np.arange(df_sub.shape[0])

for c1, ansatz_sname in enumerate(["uccsd", "he"]):
    for c2, gate_type in enumerate(gates):
        try:
            x = [i[f"gate_stats_{ansatz_sname}"][gate_type] for _, i in df_sub.iterrows()]
        except:
            x = [0 for _, i in df_sub.iterrows()]
        axs[c1].bar(xs+c2*width, x, width=width, label=gate_type.split("_")[1], color="C%i" % c2, edgecolor='k')
        print(gate_type, x, f"({ansatz_sname})")

    axs[c1].set_yscale("log")
    axs[c1].set_ylabel(f"Gate Count \n({ansatz_name_sl[ansatz_sname]})")
    
#Get tapering circuit data
n_qubits_taper = [i["taper_output"]["n_qubits_taper"] for _, i in df_sub[df_sub["active_space_method"] == i].iterrows()]
circuit_stats = [qpfas.chemistry.hardware_ansatz_gate_stats(i, 2) for i in n_qubits_taper]
df_taper = pd.DataFrame(circuit_stats)
df_taper["gate_rx"] = np.NaN
        
for c2, gate_type in enumerate(gates):
    axs[1].bar(xs+c2*width, df_taper[gate_type], width=width, hatch="////", color="C%i" % c2, edgecolor='k')

    
axs[0].legend()

axs[1].set_xticks(xs+width*(len(gates)-1)*0.5)

xlabel = [full_active_name[i] for i in as_method]
axs[1].set_xticklabels(xlabel, fontsize=8)
axs[1].set_xlabel("Active Space")

# # #fig.suptitle(nice_name(mol))

plt.savefig(f"cr_gates_{mol}.pdf")
plt.show()

In [ ]:
for ansatz_method in ["uccsd", "he"]:
    n_qubits = [i[f"gate_stats_{ansatz_method}"]["n_qubits"] for _, i in exp_df.iterrows()]
    gate_total = [i[f"gate_stats_{ansatz_method}"]["gate_total"] for _, i in exp_df.iterrows()]
    
    
    plt.scatter(n_qubits, gate_total, label=ansatz_name_sl[ansatz_method])

xs = np.linspace(4, 30, 100)
ys = [qpfas.chemistry.hardware_ansatz_gate_stats(i, 2)["gate_total"] for i in xs]
plt.plot(xs, ys, c="C1", ls='--')

plt.yscale("log")

plt.xlabel("Num. of Qubits")
plt.ylabel("Total Gate Count")

plt.legend()

plt.savefig("cr_qubitsvgates.pdf")

plt.show()


